In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA

In [4]:
# Inverse transform predictions
lstm_preds = scaler.inverse_transform(lstm_preds)
gru_preds = scaler.inverse_transform(gru_preds)
lstm_gru_preds = scaler.inverse_transform(lstm_gru_preds)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Function to calculate RMSE, MSE, R²
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mse, r2

metrics = {
    "LSTM": calculate_metrics(y_test_actual, lstm_preds),
    "GRU": calculate_metrics(y_test_actual, gru_preds),
    "LSTM+GRU": calculate_metrics(y_test_actual, lstm_gru_preds)
}

# Select the best model (lowest RMSE)
best_model_name = min(metrics, key=lambda k: metrics[k][0])
best_model = {"LSTM": lstm_model, "GRU": gru_model, "LSTM+GRU": lstm_gru_model}[best_model_name]

# Tabulate results (3 decimal places)
table = [["Model", "RMSE", "MSE", "R²"]]
for model, (rmse, mse, r2) in metrics.items():
    table.append([model, f"{rmse:.3f}", f"{mse:.3f}", f"{r2:.3f}"])

print("\n📊 Model Performance Comparison:")
print(tabulate(table, headers="firstrow", tablefmt="fancy_grid"))

# Plot predictions vs actual values
plt.figure(figsize=(14, 7))
plt.plot(df.index[seq_length:][train_size:], y_test_actual, label='Actual Price', color='black')
plt.plot(df.index[seq_length:][train_size:], lstm_preds, label=f'LSTM (RMSE: {metrics["LSTM"][0]:.3f})',
         linestyle='dashed')
plt.plot(df.index[seq_length:][train_size:], gru_preds, label=f'GRU (RMSE: {metrics["GRU"][0]:.3f})',
         linestyle='dotted')
plt.plot(df.index[seq_length:][train_size:], lstm_gru_preds, label=f'LSTM+GRU (RMSE: {metrics["LSTM+GRU"][0]:.3f})',
         linestyle='solid')

plt.legend()
plt.title('Stock Price Prediction with LSTM, GRU, & LSTM+GRU')
plt.xlabel("Date")
plt.ylabel("Price")
plt.xticks(rotation=45)
plt.show()

# Plot model loss (epochs)
plt.figure(figsize=(12, 5))
plt.plot(lstm_history.history['loss'], label="LSTM Loss")
plt.plot(gru_history.history['loss'], label="GRU Loss")
plt.plot(lstm_gru_history.history['loss'], label="LSTM+GRU Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.title("Model Loss per Epoch")
plt.show()

# Predict next 3 months using the best model
future_steps = 1000
last_sequence = X[-1]  # Use last known sequence

future_preds = []
for _ in range(future_steps):
    prediction = best_model.predict(last_sequence.reshape(1, seq_length, 1))
    future_preds.append(prediction[0, 0])
    last_sequence = np.roll(last_sequence, -1)  # Shift left
    last_sequence[-1] = prediction  # Append new prediction

# Inverse transform future predictions
future_preds = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1))

# Generate future dates
future_dates = pd.date_range(df.index[-1], periods=future_steps+1, freq='D')[1:]

# Plot future predictions
plt.figure(figsize=(14, 7))
plt.plot(df.index, df["Close"], label="Historical Prices", color='black')
plt.plot(future_dates, future_preds, label=f"Predicted Future ({best_model_name})", color='red', linestyle="dashed")
plt.legend()
plt.title(f'Future 100-days Stock Price Prediction ({best_model_name})')
plt.xlabel("Date")
plt.ylabel("Price")
plt.xticks(rotation=45)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'Historicaldata.csv'

In [ ]:
# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

# Create sequences
seq_length = 60
X, y = [], []
for i in range(len(scaled_data) - seq_length):
    X.append(scaled_data[i:i+seq_length])
    y.append(scaled_data[i+seq_length])
X, y = np.array(X), np.array(y)

# Split data
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]


In [ ]:
# Train LSTM
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32)
lstm_preds = lstm_model.predict(X_test)

# Train GRU
gru_model = Sequential([
    GRU(50, return_sequences=True, input_shape=(seq_length, 1)),
    GRU(50),
    Dense(1)
])
gru_model.compile(optimizer='adam', loss='mse')
gru_model.fit(X_train, y_train, epochs=10, batch_size=32)
gru_preds = gru_model.predict(X_test)

# Train LSTM+GRU
lstm_gru_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    GRU(50),
    Dense(1)
])
lstm_gru_model.compile(optimizer='adam', loss='mse')
lstm_gru_model.fit(X_train, y_train, epochs=10, batch_size=32)
lstm_gru_preds = lstm_gru_model.predict(X_test)


In [ ]:
# Train ARIMA
train_data = df.iloc[:train_size]['Close']
test_data = df.iloc[train_size:]['Close']

arima_model = ARIMA(train_data, order=(5,1,0))  # Order can be tuned
darima_model = arima_model.fit()
arima_preds = darima_model.forecast(steps=len(test_data))
arima_preds = np.array(arima_preds).reshape(-1, 1)


In [ ]:
# Inverse transform predictions
lstm_preds = scaler.inverse_transform(lstm_preds)
gru_preds = scaler.inverse_transform(gru_preds)
lstm_gru_preds = scaler.inverse_transform(lstm_gru_preds)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))


In [ ]:
# Calculate RMSE and MSE
lstm_rmse = np.sqrt(mean_squared_error(y_test_actual, lstm_preds))
lstm_mse = mean_squared_error(y_test_actual, lstm_preds)
gru_rmse = np.sqrt(mean_squared_error(y_test_actual, gru_preds))
gru_mse = mean_squared_error(y_test_actual, gru_preds)
lstm_gru_rmse = np.sqrt(mean_squared_error(y_test_actual, lstm_gru_preds))
lstm_gru_mse = mean_squared_error(y_test_actual, lstm_gru_preds)
arima_rmse = np.sqrt(mean_squared_error(test_data, arima_preds))
arima_mse = mean_squared_error(test_data, arima_preds)


In [ ]:
# Combine train and test predictions
full_dates = df.index[seq_length:]
full_actual = np.concatenate([y_train_actual, y_test_actual])
full_lstm = np.concatenate([np.full_like(y_train_actual, np.nan), lstm_preds])
full_gru = np.concatenate([np.full_like(y_train_actual, np.nan), gru_preds])
full_lstm_gru = np.concatenate([np.full_like(y_train_actual, np.nan), lstm_gru_preds])
full_arima = np.concatenate([np.full_like(y_train_actual, np.nan), arima_preds])

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(full_dates, full_actual, label='Actual Price', color='black')
plt.plot(full_dates, full_lstm, label=f'LSTM (RMSE: {lstm_rmse:.2f}, MSE: {lstm_mse:.2f})', linestyle='dashed')
plt.plot(full_dates, full_gru, label=f'GRU (RMSE: {gru_rmse:.2f}, MSE: {gru_mse:.2f})', linestyle='dotted')
plt.plot(full_dates, full_lstm_gru, label=f'LSTM+GRU (RMSE: {lstm_gru_rmse:.2f}, MSE: {lstm_gru_mse:.2f})', linestyle='solid')
plt.plot(full_dates, full_arima, label=f'ARIMA (RMSE: {arima_rmse:.2f}, MSE: {arima_mse:.2f})', linestyle='dashdot')
plt.legend()
plt.title('Stock Price Prediction with LSTM, GRU, LSTM+GRU, and ARIMA')
plt.xlabel("Date")
plt.ylabel("Price")
plt.xticks(rotation=45)
plt.show()
